In [235]:
import pandas as pd
import numpy as np
import datetime


from sklearn.ensemble import RandomForestClassifier

In [236]:
input_fire_csv_path = '../data/FW_Veg_Rem_Combined.csv'
input_city_csv_path =  '../data/city_attributes.csv'
input_humidity_csv_path = '../data/humidity.csv'
input_pressure_csv_path = '../data/pressure.csv'
input_temperature_csv_path = '../data/temperature.csv'
input_windD_csv_path = '../data/wind_direction.csv'
input_windS_csv_path =  '../data/wind_speed.csv'

In [237]:
df_fire = pd.read_csv(input_fire_csv_path)
df_city = pd.read_csv(input_city_csv_path)
df_hum = pd.read_csv(input_humidity_csv_path)
df_pre = pd.read_csv(input_pressure_csv_path)
df_tem = pd.read_csv(input_temperature_csv_path)
df_windD = pd.read_csv(input_windD_csv_path)
df_windS = pd.read_csv(input_windS_csv_path)

In [238]:
df_fire

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,2,NaN,60.0,C,Arson,34.947800,-88.722500,MS,2/29/2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,3,WNA 1,1.0,B,Debris Burning,39.641400,-119.308300,NV,6/6/2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,4,NaN,2.0,B,Miscellaneous,30.700600,-90.591400,LA,9/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55362,55362,55362,IMPACT,6289.0,G,Missing/Undefined,39.180000,-96.784167,KS,10/15/2015,...,3.804803,65.671410,61.839572,54.625698,55.042092,35.4,8.2,0.0,249.0,0.331501
55363,55363,55363,BUTTE,70868.0,G,Missing/Undefined,38.342719,-120.695967,CA,9/9/2015,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.124683
55364,55364,55364,WILLOW,5702.0,G,Arson,37.262607,-119.511139,CA,7/25/2015,...,2.529158,43.755556,43.755556,44.443975,35.924406,0.0,0.0,0.0,0.0,0.097682
55365,55365,55365,BARKER,3261.0,F,Miscellaneous,40.604300,-123.080450,CA,7/31/2015,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.178206


In [239]:
df_dic = {  'humidity':df_hum,
            'pressure':df_pre,
            'temperature':df_tem,
            'wind_direction':df_windD,
            'wind_speed':df_windS}

In [240]:
for key in df_dic.keys():
    df_dic[key]['datetime'] = pd.to_datetime(df_dic[key]['datetime']).apply(datetime.datetime.date)
    df_dic[key] = df_dic[key].groupby('datetime').mean()
    df_dic[key] = df_dic[key].stack(0).reset_index()
    df_dic[key].rename(columns={'datetime':'date','level_1':'city', 0:key}, inplace=True)
df_hum, df_pre, df_tem, df_windD, df_windS = df_dic.values()

In [241]:
df_final = df_hum
for key in df_dic.keys():
    if key == 'humidity':
        continue
    else:
        df_final=df_final.merge(df_dic[key], on=['date', 'city'])
df_final

,date,city,humidity,pressure,temperature,wind_direction,wind_speed
0,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000
1,2012-10-01,San Francisco,83.000000,1009.727273,289.416642,122.363636,1.636364
2,2012-10-01,Seattle,78.000000,1030.000000,281.767262,32.272727,0.000000
3,2012-10-01,Los Angeles,88.000000,1013.000000,291.846501,0.000000,0.000000
4,2012-10-01,San Diego,79.909091,1013.000000,291.573495,0.000000,0.000000
...,...,...,...,...,...,...,...
67533,2017-11-30,Pittsburgh,42.000000,1025.000000,279.190000,360.000000,2.000000
67534,2017-11-30,Toronto,60.000000,1027.000000,274.510000,330.000000,3.000000
67535,2017-11-30,Philadelphia,32.000000,1024.000000,283.420000,360.000000,4.000000
67536,2017-11-30,Montreal,58.000000,1027.000000,271.800000,300.000000,4.000000


In [242]:
df_city.rename(columns={'City':'city'}, inplace=True)

In [243]:
df_final = df_final.merge(df_city, on=['city'])
df_final

,date,city,humidity,pressure,temperature,wind_direction,wind_speed,Country,Latitude,Longitude
0,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208
1,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208
2,2012-10-03,Portland,66.208333,1021.083333,289.599792,228.333333,2.625000,United States,45.523449,-122.676208
3,2012-10-04,Portland,51.166667,1022.875000,286.482500,206.750000,4.625000,United States,45.523449,-122.676208
4,2012-10-05,Portland,40.391304,1022.916667,288.286042,182.250000,3.708333,United States,45.523449,-122.676208
...,...,...,...,...,...,...,...,...,...,...
67533,2017-10-24,Vancouver,84.208333,1035.458333,283.835417,139.333333,1.708333,Canada,49.249660,-123.119339
67534,2017-10-25,Vancouver,92.583333,1021.166667,283.138333,156.583333,2.666667,Canada,49.249660,-123.119339
67535,2017-10-26,Vancouver,93.208333,1024.166667,283.408333,310.958333,3.166667,Canada,49.249660,-123.119339
67536,2017-10-27,Vancouver,93.583333,1024.291667,283.085417,281.291667,2.125000,Canada,49.249660,-123.119339


In [244]:
df_fire = df_fire[['disc_clean_date', 'latitude', 'longitude']]
df_fire['disc_clean_date']=pd.to_datetime(df_fire['disc_clean_date']).apply(datetime.datetime.date)
df_fire.rename(columns={'latitude':'Latitude', 'longitude':'Longitude', 'disc_clean_date':'date'}, inplace=True)
df_fire

<ipython-input-244-32e8a4d12645>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fire['disc_clean_date']=pd.to_datetime(df_fire['disc_clean_date']).apply(datetime.datetime.date)
c:\Users\julian.koechlin\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,date,Latitude,Longitude
0,2007-02-11,18.105072,-66.753044
1,2006-12-11,35.038330,-87.610000
2,2004-02-29,34.947800,-88.722500
3,2005-06-06,39.641400,-119.308300
4,1999-09-22,30.700600,-90.591400
...,...,...,...
55362,2015-10-15,39.180000,-96.784167
55363,2015-09-09,38.342719,-120.695967
55364,2015-07-25,37.262607,-119.511139
55365,2015-07-31,40.604300,-123.080450


In [245]:
df_final_fire_merge = df_final.merge(df_fire, on=['date'], how='left')

In [246]:
df_final_fire_merge

,date,city,humidity,pressure,temperature,wind_direction,wind_speed,Country,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208,42.432643,-94.575291
1,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208,45.689806,-93.979184
2,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208,47.870300,-94.984200
3,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208,48.385440,-119.441500
4,2012-10-03,Portland,66.208333,1021.083333,289.599792,228.333333,2.625000,United States,45.523449,-122.676208,43.354120,-116.240780
...,...,...,...,...,...,...,...,...,...,...,...,...
242304,2017-10-24,Vancouver,84.208333,1035.458333,283.835417,139.333333,1.708333,Canada,49.249660,-123.119339,NaN,NaN
242305,2017-10-25,Vancouver,92.583333,1021.166667,283.138333,156.583333,2.666667,Canada,49.249660,-123.119339,NaN,NaN
242306,2017-10-26,Vancouver,93.208333,1024.166667,283.408333,310.958333,3.166667,Canada,49.249660,-123.119339,NaN,NaN
242307,2017-10-27,Vancouver,93.583333,1024.291667,283.085417,281.291667,2.125000,Canada,49.249660,-123.119339,NaN,NaN


In [247]:
df_final_fire_merge['Longitude_diff'] = np.abs(df_final_fire_merge['Longitude_x']-df_final_fire_merge['Longitude_y'])
df_final_fire_merge['Latitude_diff'] = np.abs(df_final_fire_merge['Latitude_x']-df_final_fire_merge['Latitude_y'])
df_final_fire_merge

,date,city,humidity,pressure,temperature,wind_direction,wind_speed,Country,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Longitude_diff,Latitude_diff
0,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208,42.432643,-94.575291,28.100917,3.090806
1,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208,45.689806,-93.979184,28.697024,0.166357
2,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208,47.870300,-94.984200,27.692008,2.346851
3,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208,48.385440,-119.441500,3.234708,2.861991
4,2012-10-03,Portland,66.208333,1021.083333,289.599792,228.333333,2.625000,United States,45.523449,-122.676208,43.354120,-116.240780,6.435428,2.169329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242304,2017-10-24,Vancouver,84.208333,1035.458333,283.835417,139.333333,1.708333,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN
242305,2017-10-25,Vancouver,92.583333,1021.166667,283.138333,156.583333,2.666667,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN
242306,2017-10-26,Vancouver,93.208333,1024.166667,283.408333,310.958333,3.166667,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN
242307,2017-10-27,Vancouver,93.583333,1024.291667,283.085417,281.291667,2.125000,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN


In [248]:
df_final_fire_merge['close_enough_to_count']=(df_final_fire_merge['Longitude_diff']<=2)&(df_final_fire_merge['Latitude_diff']<=2)


In [249]:
df_final_fire_merge

,date,city,humidity,pressure,temperature,wind_direction,wind_speed,Country,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Longitude_diff,Latitude_diff,close_enough_to_count
0,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208,42.432643,-94.575291,28.100917,3.090806,False
1,2012-10-01,Portland,78.727273,1024.000000,282.118197,57.727273,0.000000,United States,45.523449,-122.676208,45.689806,-93.979184,28.697024,0.166357,False
2,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208,47.870300,-94.984200,27.692008,2.346851,False
3,2012-10-02,Portland,65.833333,1023.583333,286.137728,214.041667,1.291667,United States,45.523449,-122.676208,48.385440,-119.441500,3.234708,2.861991,False
4,2012-10-03,Portland,66.208333,1021.083333,289.599792,228.333333,2.625000,United States,45.523449,-122.676208,43.354120,-116.240780,6.435428,2.169329,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242304,2017-10-24,Vancouver,84.208333,1035.458333,283.835417,139.333333,1.708333,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN,False
242305,2017-10-25,Vancouver,92.583333,1021.166667,283.138333,156.583333,2.666667,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN,False
242306,2017-10-26,Vancouver,93.208333,1024.166667,283.408333,310.958333,3.166667,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN,False
242307,2017-10-27,Vancouver,93.583333,1024.291667,283.085417,281.291667,2.125000,Canada,49.249660,-123.119339,NaN,NaN,NaN,NaN,False


In [250]:
df_final_fire_merge.to_csv('../data/clean_dataset.csv', index=False)